In [5]:
import os
import requests
import tempfile
from langchain import  PromptTemplate
from pinecone import Pinecone
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings  
from langchain_community.llms import Ollama
from langchain.vectorstores import Pinecone
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama.llms import OllamaLLM
from langchain_groq import ChatGroq
import dotenv

In [6]:
ROOT_DIR=os.path.abspath('..')

In [11]:
PINECONE_INDEX_NAME = "main-project"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [12]:
def load_data(data_path):
    print(data_path)
    loader=PyPDFDirectoryLoader(data_path)
    data =loader.load()
    return data

docs = load_data(ROOT_DIR)
len(docs)

c:\Users\chhan\OneDrive\Desktop\BOT


759

In [13]:
def text_split(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap=200)
    text_chunks = splitter.split_documents(data)
    return text_chunks

text_chunks =text_split(docs)
print(len(text_chunks))

10289


In [14]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
os.environ['PINECONE_API_KEY']="pcsk_2v4NVt_PCxDtbj9z3ABZymMMvbQ7mUbsut1adBZVLvVRZdAAAoHhLrfsNGMWosfDMGAg5"
docsearch = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME,embedding=embeddings)
print("index successfully loaded")

index successfully loaded


In [15]:
vector= embeddings.embed_query("what is vitamins?")
len(vector)

384

In [16]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
    text_chunks,
    index_name=PINECONE_INDEX_NAME,
    embedding= embeddings
)

In [17]:
docsearch= PineconeVectorStore.from_existing_index(PINECONE_INDEX_NAME,embeddings)

In [18]:
query= "What is diabetics?"
docs = docsearch.similarity_search(query, k=3)
print(docs)

[Document(metadata={'page': 444.0, 'source': 'c:\\Users\\chhan\\OneDrive\\Desktop\\BOT\\Code\\medical_book.pdf'}, page_content='Diabetic neuropathy\nDefinition\nDiabetic neuropathy is a nerve disorder caused by\ndiabetes mellitus . Diabetic neuropathy may be diffuse,\naffecting several parts of the body, or focal, affecting a\nspecific nerve and part of the body.\nDescription\nThe nervous system consists of two major divisions:\nthe central nervous systems (CNS) which includes the\nbrain, the cranial nerves, and the spinal cord, and the\nperipheral nervous system (PNS) which includes the'), Document(metadata={'page': 435.0, 'source': 'c:\\Users\\chhan\\OneDrive\\Desktop\\BOT\\Code\\medical_book.pdf'}, page_content='glucose in the blood cannot be absorbed into the cells of\nthe body. Symptoms include frequent urination, lethargy,\nexcessive thirst, and hunger. The treatment includes\nchanges in diet, oral medications, and in some cases,\ndaily injections of insulin.\nDescription\nDiabet

In [19]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k":10})


In [20]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key="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",
      model_name="llama3-8b-8192"
      )

In [21]:
llm= OllamaLLM(model="llama2")


In [22]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt= (
    "You are an assistant for question-answer task."
    "Use the given information to answer the questions."
    "If you don't know the answer , say you don't know."
    "Do not generate questions , just answer the provided question"
    "Try to give accurate answers"
   "{context}" 
)

prompt = ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt),
    ("human","{input}")

    ]
)

In [23]:
QAChain= create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever,QAChain)

In [ ]:
response=rag_chain.invoke({"input":"What is vitamins?"})
print(response["answer"])

In [28]:
response=rag_chain.invoke({"input":"How much vitamin c is required for a person per day"})
print(response["answer"])

Based on the information provided in the passage, the recommended daily dose of vitamin C for a healthy adult is 400-500 mg per day. However, if a person is experiencing constipation, the passage suggests that an increase in dosage to 1,000 mg per day may be helpful in preventing or relieving symptoms. Additionally, the passage mentions that elective surgery is discouraged due to the high possibility of complications, and alternative treatment options such as constitutional homeopathic treatment may be helpful in maintaining optimal health in persons with a diagnosis of EDS (Ehlers-Danlos syndrome).


In [51]:
def data_load_pdf(file):
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_file_path = f"{temp_dir}/{file.name}"
        with open(temp_file_path, "wb") as temp_file:
            temp_file.write(file.read())

        loader = DirectoryLoader(temp_dir, glob="*.pdf", loader_cls=PyPDFLoader)
        data = loader.load()
        return data


In [72]:
import chroma

vector_store = chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

SyntaxError: multiple exception types must be parenthesized (core.py, line 273)

In [76]:
def data_load_pdf(file):
    if file is not None:
        text=""
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
        documents = [Document(page_content=text)]
    training(documents)

data_load_pdf(medical_book.pdf)

NameError: name 'medical_book' is not defined

In [73]:
import PyPDF2